In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import MisrToolkit as mtk # needs python3.6
from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob

# Extract data from MISR-estimated roughness files by using MTK library. Should use python3.6 for this purpose 

## Plot Transect 

In [1]:
roughness_dir = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH"


In [2]:
# extract MISR-estimated roughness for the same latLon locations and plot them together with ATM
transect_from_misr = []
for irow in range(transect_small.shape[0]): # each row==LatLon might fall in several blocks==POB, we open each block and will average all at the end
    
    print("DF row: %s" %irow)
    lat_lon = transect_small[['lat','lon']].iloc[irow]
    print(lat_lon)
    
    path_list = mtk.latlon_to_path_list(lat_lon[0], lat_lon[1])
    print(path_list)
    
    roughness_per_pixel_list = []
    for path in path_list:
        # Q- now that we have found many paths, how figure out to select which path?
        bls = mtk.latlon_to_bls(path, 275 , lat_lon[0], lat_lon[1])
        print(bls)
        
        # use P and B to find roughness file, open it, and extract value for LS, append it to a list, and maybe finally append the list to DF as the last column
        if path < 100:
            print(path)
            path_str = str(path).zfill(3)
            print(path_str)
            
        roughness_filepattern = "roughness_toa_refl_P"+path_str+"_*"+"_B0"+str(bls[0])+".dat"
        print(roughness_filepattern)
        rough_file_fp = os.path.join(roughness_dir, roughness_filepattern)
        print(rough_file_fp)
        
        # check if file is found on machine
        roughness_file_found_list = glob.glob(os.path.join(roughness_dir, roughness_filepattern))
        print("if more than 1 file found, there is a problem with this code and it has to be reviewed")
        print(len(roughness_file_found_list))
        
        if (len(roughness_file_found_list) <= 0):
            print("roughness file not found- continue to next path")
            continue
        # open roughness file and read pixel value- chat binary format to 2D array and read pixel value
        rough_2d_arr = np.fromfile(roughness_file_found_list[0], dtype=np.double)[0:1048576].reshape((512,-1))
        print(rough_2d_arr.shape)
        
        pixel_row = bls[1]
        pixel_col = bls[2]
        roughness_value = rough_2d_arr[int(pixel_row), int(pixel_col)]
        if roughness_value > 0:
            roughness_per_pixel_list.append(roughness_value)
    
    print("----------> rough pixels...")
    print(roughness_per_pixel_list)
    mean_roughness_per_pixel = sum(roughness_per_pixel_list)/len(roughness_per_pixel_list)
    
    transect_from_misr.append(mean_roughness_per_pixel)


NameError: name 'transect_small' is not defined

In [ ]:
# plot transect
transect_small["rms"].plot(x=transect_small.index)#, y=transect_small)
plt.plot(transect_from_misr)

In [ ]:
transect_from_misr

## statistics

In [ ]:
transect_small.describe()

## histogram

In [ ]:
transect_small["rms"].hist(bins=20, alpha=1)